In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, LeakyReLU, BatchNormalization, Dropout, Concatenate
from tensorflow.keras import Input, Model

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = 'D:\icecube\sim_data'

# Name for model
key = 'OffBrandons model'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':'product', 't':False, 'normed':False, 'reco' : 'plane' , 'cosz' : False}#cosz = True}

x, y = load_preprocessed(simPrefix, 'train') #filter nans out before training!
#print(y.keys())

Percentage of events with a NaN: 2.68


In [3]:
    ip1 = Input(shape=(10,10,1))
    l = Conv2D(64,kernel_size=3,padding='same',use_bias=False)(ip1)
    l = BatchNormalization()(l)
    l = LeakyReLU(alpha=.3)(l)
    l = Conv2D(32,kernel_size=3,padding='same',use_bias=False)(l)
    l = BatchNormalization()(l)
    l = LeakyReLU(alpha=.3)(l)
    l = Dropout(.6)(l)
    l = Flatten()(l)
    ip2 = Input(shape= (1,))
    merge = Concatenate()([l,ip2])
    l = Dense(1,use_bias=False)(l)
    l = BatchNormalization()(l)
    l = LeakyReLU(alpha=.3)(l)
    l = Dropout(.5)(l)
    output = Dense(1)
    
    model = Model(inputs = [ip1,ip2], outputs = output,name=nameModel(prep,prefix='test'))
    
    model.summary()

    model.compile(loss='mean_squared_error', optimizer='SGD', metrics=['mse'])

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: <keras.layers.core.dense.Dense object at 0x000002AA316CB430>

In [ ]:


# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)

energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

e_y = energy
comp_y = y['comp']


           



In [ ]:


history = model.fit([x_i[0], x_i[1]] ,e_y, validation_split = .15, epochs=3, batch_size = 64)




In [7]:
model.save('%s.h5' % key)

np.save('%s.h5' % key, prep)